In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 108MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 5

## Base Model 5

In [ ]:
base_5 = pd.read_csv('/content/Eval/model_base_5/content/eval_basemodel_20250525_073715/base_model_outputs.csv')
base_5.head(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","system\nKamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.\nuser\nInstruksi: Solusi atasi sakit maag yang tak kunjung sembuh\nPertanyaan: Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosany...",0.685527,0.67966,0.682581


bershihin dulu generated

In [ ]:
def get_text_after_assistant(text):
  """
  Mengambil teks setelah kata 'assistant' dalam sebuah string.

  Args:
    text: String input (diasumsikan string, tangani NaN jika perlu).

  Returns:
    String teks setelah 'assistant', atau None jika 'assistant' tidak ditemukan atau input bukan string.
  """
  if not isinstance(text, str): # Menangani jika ada nilai non-string (misalnya NaN)
    return None
  parts = text.split("assistant")
  if len(parts) > 1:
    return parts[1].strip()
  else:
    return None
base_5['generated'] = base_5['generated'].apply(get_text_after_assistant)

In [ ]:
base_5.head(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Maaf, saya tidak bisa memberikan diagnosa langsung atau saran pengobatan. Namun, saya dapat memberikan beberapa informasi umum tentang sakit maag dan saran yang mungkin membantu.\n\nSakit maag atau dispepsia adalah kondisi yang umum terjadi dan dapat disebabkan oleh berbagai faktor, seperti pola makan yang tidak sehat, stres, atau gangguan pencernaan. Gejala sakit maag yang Anda deskripsikan, seperti nyeri ulu hati, sendawa, dan muntah, dapat terjadi pada penderita maag.\n\nNamun, jika gejal...",0.685527,0.67966,0.682581


In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_5.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Maaf, saya tidak bisa memberikan diagnosa langsung atau saran pengobatan. Namun, saya dapat memberikan beberapa informasi umum tentang sakit maag dan saran yang mungkin membantu.\n\nSakit maag atau dispepsia adalah kondisi yang umum terjadi dan dapat disebabkan oleh berbagai faktor, seperti pola makan yang tidak sehat, stres, atau gangguan pencernaan. Gejala sakit maag yang Anda deskripsikan, seperti nyeri ulu hati, sendawa, dan muntah, dapat terjadi pada penderita maag.\n\nNamun, jika gejal...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670534,0.631100,0.650220
1,"Jika feses Anda cair tapi sulit keluar, kemungkinan besar Anda mengalami sembelit. Sembelit dapat disebabkan oleh beberapa faktor, seperti kurangnya serat dalam makanan, kurangnya cairan, atau stres.\n\nBeberapa solusi untuk mengatasi sembelit adalah:\n\n1. **Makan makanan kaya serat**: Konsumsi makanan kaya serat seperti buah-buahan, sayuran, dan biji-bijian. Ini dapat membantu melunakkan feses dan memudahkan proses buang air besar.\n2. **Minum banyak air**: Pastikan Anda minum banyak air u...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.652869,0.651373,0.652120
2,"Sakit perut, mual, dan bersendawa berlebihan dapat disebabkan oleh beberapa kondisi, seperti:\n\n* **Gastroesophageal Reflux Disease (GERD):** Penyakit yang disebabkan oleh refluks asam lambung ke kerongkongan, yang dapat menyebabkan perut sakit, mual, dan bersendawa.\n\n* **Infeksi lambung:** Infeksi bakteri atau virus dapat menyebabkan sakit perut, mual, dan diare.\n\n* **Sindrom iritasi usus besar (IBS):** Kondisi yang menyebabkan perubahan pada pola buang air besar dan dapat menyebabkan ...","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...",0.672381,0.666963,0.669661
3,"Sakit perut yang disertai mual dan muntah bisa disebabkan oleh berbagai kondisi. Berdasarkan gejala yang kamu alami, beberapa kemungkinan penyebabnya antara lain:\n\n1. **Gangguan pencernaan**: Masalah pencernaan seperti gastritis (peradangan pada lapisan lambung), tukak lambung (luka pada lapisan lambung), atau irritable bowel syndrome (IBS) bisa menyebabkan sakit perut dan mual.\n\n2. **Infeksi saluran pencernaan**: Infeksi bakteri seperti gastroenteritis atau infeksi lainnya dapat menyeba...","sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala yang anda alami dan melakukan pemeriksaan fisik yang diperlukan. pemeriksaan penunjang seperti tes darah, tes urine, usg mungkin akan dokter anjurkan. terapi yang sesuai akan dokter berikan.anjuran yang dapat anda lakukan: konsumsi makanan bersih dan bergizi

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Maaf, saya tidak bisa memberikan diagnosa langsung atau saran pengobatan. Namun, saya dapat memberikan beberapa informasi umum tentang sakit maag dan saran yang mungkin membantu.\n\nSakit maag atau dispepsia adalah kondisi yang umum terjadi dan dapat disebabkan oleh berbagai faktor, seperti pola makan yang tidak sehat, stres, atau gangguan pencernaan. Gejala sakit maag yang Anda deskripsikan, seperti nyeri ulu hati, sendawa, dan muntah, dapat terjadi pada penderita maag.\n\nNamun, jika gejal...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670534,0.631100,0.65022,27.044806,10.938439,5.336597,2.638988,33.408578,8.163265,14.898420,20.316027
1,"Jika feses Anda cair tapi sulit keluar, kemungkinan besar Anda mengalami sembelit. Sembelit dapat disebabkan oleh beberapa faktor, seperti kurangnya serat dalam makanan, kurangnya cairan, atau stres.\n\nBeberapa solusi untuk mengatasi sembelit adalah:\n\n1. **Makan makanan kaya serat**: Konsumsi makanan kaya serat seperti buah-buahan, sayuran, dan biji-bijian. Ini dapat membantu melunakkan feses dan memudahkan proses buang air besar.\n2. **Minum banyak air**: Pastikan Anda minum banyak air u...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.652869,0.651373,0.65212,29.651163,14.424921,8.490162,4.362298,33.212996,8.727273,15.162455,23.104693


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.681171
bert_recall,0.652582
bert_f1,0.666140
bleu1,18.572810
bleu2,7.586095
bleu3,3.901354
bleu4,2.448177
rouge1,25.707750
rouge2,4.680531
rougeL,13.407218


In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_5_mean.csv')

In [ ]:
df_scores = pd.concat([base_5[['input']], df_scores], axis=1)
df_scores.head(2)

,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Maaf, saya tidak bisa memberikan diagnosa langsung atau saran pengobatan. Namun, saya dapat memberikan beberapa informasi umum tentang sakit maag dan saran yang mungkin membantu.\n\nSakit maag atau dispepsia adalah kondisi yang umum terjadi dan dapat disebabkan oleh berbagai faktor, seperti pola makan yang tidak sehat, stres, atau gangguan pencernaan. Gejala sakit maag yang Anda deskripsikan, seperti nyeri ulu hati, sendawa, dan muntah, dapat terjadi pada penderita maag.\n\nNamun, jika gejal...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670534,0.631100,0.65022,27.044806,10.938439,5.336597,2.638988,33.408578,8.163265,14.898420,20.316027
1,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","Jika feses Anda cair tapi sulit keluar, kemungkinan besar Anda mengalami sembelit. Sembelit dapat disebabkan oleh beberapa faktor, seperti kurangnya serat dalam makanan, kurangnya cairan, atau stres.\n\nBeberapa solusi untuk mengatasi sembelit adalah:\n\n1. **Makan makanan kaya serat**: Konsumsi makanan kaya serat seperti buah-buahan, sayuran, dan biji-bijian. Ini dapat membantu melunakkan feses dan memudahkan proses buang air besar.\n2. **Minum banyak air**: Pastikan Anda minum banyak air u...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.652869,0.651373,0.65212,29.651163,14.424921,8.490162,4.362298,33.212996,8.727273,15.162455,23.104693


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_5.csv')

## Fine Tune Model 1

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_5 = pd.read_csv('/content/Eval/model_ft_5/content/eval_basemodel_20250525_083515/fine_tuned_outputs.csv')
ft_model_5.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
80,Sakit perut seperti di tusuk-tusuk setelah minum obat pereda mual saat hamil,"Sakit perut seperti di tusuk-tusuk setelah minum obat pereda mual saat hamil. Dok , saya mau bertanya dong hari ini teman saya habis usg dan alhamdulilah teman saya hamil lalu teman saya diberi obt pereda mual 30 menit diminum sebelum makan . 5 menit setelah minum obt itu teman saya merasa perutnya sakit banget seperti di tusuk-tusuk dok kayak asam lambung naik tapi saya makan mulu beberapa menit kemudian sakit perutnya reda tapi pindah ke pinggul sebelah kiri dan rasanya panas Pertanyaannya...","nyeri perut tersebut dapat disebabkan oleh: cedera otot gastritis tukak lambung penyakit asam lambung batu empedu infeksi saluran kemih infeksi ginjal batu ginjal ancaman keguguran dsb. sebaiknya konsultasikan keluhan anda dengan dokter spesialis penyakit dalam. dokter akan menelusuri riwayat gejala yang anda alami dan melakukan pemeriksaan fisik. tes penunjang seperti tes darah, tes urine, usg dapat dokter anjurkan. terapi yang tepat akan dokter berikan. anjuran yang bisa anda lakukan: hind...","system\nKamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.\nuser\nInstruksi: Sakit perut seperti di tusuk-tusuk setelah minum obat pereda mual saat hamil\nPertanyaan: Sakit perut seperti di tusuk-tusuk setelah minum obat pereda mual saat hamil. Dok, saya mau bertanya dong hari ini teman saya habis usg dan alha...",0.663558,0.712541,0.687178


In [ ]:
# Ini akan membuat kolom baru bernama 'assistant_response' dengan hasil ekstraksi
ft_model_5['generated'] = ft_model_5['generated'].apply(get_text_after_assistant)

In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_5)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,10.808236,5.263204,2.878571,1.793664,21.808511,5.347594,9.574468,9.574468,69.060004,64.247406,66.566831,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",saya akan coba bantu jawab. keluhan sakit maag atau yang dalam istilah medis disebut dispepsia merupakan suatu keadaan dimana timbulnya rasa nyeri pada ulu hati disertai dengan keluhan lainnya. berikut ini beberapa gejala yang dapat ditimbulkan oleh sakit maag diantaranya adalah : rasa tidak nyaman pada perut bagian atas rasa panas pada perut bagian atas nyeri ulu hati mual muntah kembung sering bersendawa dulu. penyebab dari timbulnya sakit maag diantaranya adalah : infeksi bakteri h. pylor...
1,32.618363,18.904514,12.291311,7.770828,36.293436,11.673152,21.621622,21.621622,72.084212,69.511414,70.774442,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"diantaranya: kurang konsumsi serat, kurang konsumsi cairan, kurang aktivitas fisik, kebiasaan menunda keinginan bab, gangguan mental seperti kecemasan, depresi, atau gangguan somatoform (tidak bisa mengontrol keinginan untuk bab), efek samping obat-obatan tertentu, gangguan pada usus, dan lain-lain. beberapa tips yang dapat anda lakukan untuk mengatasi konstipasi: perbanyak konsumsi serat dengan mengonsumsi sayur, buah, dan kacang-kacangan perbanyak konsumsi cairan, minimal 2 liter sehari ol..."


In [ ]:
mean_ft

,0
bleu1,23.161729
bleu2,11.119234
bleu3,6.203402
bleu4,3.837908
rouge1,27.610969
rouge2,6.217843
rougeL,14.467971
rougeLSum,14.684577
bert_precision,69.754717
bert_recall,67.469750


In [ ]:
ft_model_5.to_csv('Hasil Eval revisi/eval_ft_model_5.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_5_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-5.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_basemodel_5.csv (deflated 70%)
  adding: content/Hasil Eval revisi/eval_ft_model_5_mean.csv (deflated 37%)
  adding: content/Hasil Eval revisi/eval_basemodel_5_mean.csv (deflated 37%)
  adding: content/Hasil Eval revisi/eval_ft_model_5.csv (deflated 72%)
